In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from ../.env
load_dotenv(dotenv_path='../.env')

# Access the OPENAI_API_KEY
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY not found in the environment variables")


In [6]:
from gradio_client import Client, handle_file

client = Client("microsoft/OmniParser", verbose=True)
result = client.predict(
		image_input=handle_file('avanza_screenshot.png'),
		box_threshold=0.05,
		iou_threshold=0.1,
		api_name="/process"
)
print(result)




Loaded as API: https://microsoft-omniparser.hf.space ✔
('/private/var/folders/4l/xzbwfz351fb1xv7gjqv5634m0000gn/T/gradio/91fe12b92e71224f906fe87a8cb0a117ce46628e2bd5be5e1b18b3ad360b649c/image.webp', 'Text Box ID 0: Aktier\nText Box ID 1: Utveckling\nText Box ID 2: Inkopsinfoe\nText Box ID 3: Historik\nText Box ID 4: Nyckeltal\nText Box ID 5: Utdelningare\nText Box ID 6: tt Anpassa\nText Box ID 7: Land Namn \nText Box ID 8: Konto\nText Box ID 9: Antal\nText Box ID 10: 1 dag %\nText Box ID 11: Senast\nText Box ID 12: Sedan kop\nText Box ID 13: Sedan kop %.\nText Box ID 14: Varde\nText Box ID 15: Verktyge\nText Box ID 16: Kop Salj\nText Box ID 17:  Coinbase Global Ae\nText Box ID 18: ISK - Alte....\nText Box ID 19: 437\nText Box ID 20: 0,00%\nText Box ID 21: 319,13\nText Box ID 22: +214 533 kr\nText Box ID 23: +16,41%\nText Box ID 24: 1 521 507 kr\nText Box ID 25: 0\nText Box ID 26: 4\nText Box ID 27: Totalt vardee\nText Box ID 28: 0,00%\nText Box ID 29: +214 533 kr\nText Box ID 30: +16,4

In [8]:
from llama_index.core import Document
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# Initialize OpenAI
Settings.llm = OpenAI(model="gpt-3.5-turbo")

# Create a document from the result
doc = Document(text=str(result))

# Create a simple prompt to analyze the content
prompt = "Please explain what this data represents and what kind of information it contains. Data: {}"
response = Settings.llm.complete(prompt.format(str(result)))

print(response.text)

This data represents a collection of text boxes containing information related to stocks, funds, and certificates. Each text box has an ID associated with it, and the data includes details such as stock names, account information, performance metrics, and values. Additionally, there are icon boxes with functions like favorite or bookmark options, a United States flag feature, notifications, and draw functions.

The data also includes coordinates for each text box, specifying its position and size on a graphical user interface. These coordinates are represented as percentages of the total screen size. The last part of the data contains information about the position and size of various icon boxes on the interface.

Overall, this data provides structured information about financial instruments and their performance, along with details about the graphical user interface elements that display this information.


In [9]:
# Create a more structured prompt to analyze and format the data into tables
table_prompt = """
Please analyze this data and structure it into clear tables. Focus on:
1. Stock information (price, change, etc)
2. Any financial metrics present
3. Any other relevant data that can be organized

Present the information in a clear tabular format using markdown tables.

Data: {}
"""

table_response = Settings.llm.complete(table_prompt.format(str(result)))

print(table_response.text)


### Stock Information

| Stock Name             | Account Type | 1 Day % Change | Last Price | Change Since Purchase | Change Since Purchase % | Total Value |
|------------------------|--------------|----------------|------------|-----------------------|------------------------|-------------|
| Coinbase Global Ae     | ISK - Alte...| 0.00%          | 319.13     | +214 533 kr           | +16.41%                | 1 521 507 kr|
| Swedbank Robur Technology A | ISK - Fond...| +0.65%   | Igar       | +168 562 kr           | +16.86%                | 1 168 562 kr|
| Valour Bitcoin (BTC) Zero SEK | ISK - A...| +1.57%     | 96.29      | +503 556 kr           | +33.32%                | 2 014 676 kr|
| Valour Ethereum (ETH) Zero SEK | ISK - A...| -2.94%    | 34.63      | +160 456 kr           | +14.51%                | 1 266 073 kr|
| Valour Solana (SOL) SEK | ISK - A...| -2.04%         | 20.18      | +284 933 kr           | +24.11%                | 1 466 561 kr|

### Financial Metrics

- Total Va

In [16]:
# Create a prompt to extract stock tickers
ticker_prompt = """
Please analyze this data and extract all tickers of the assets mentioned on each row. 

Return the tickers that would be valid for Yahoo Finance API (e.g., AAPL, MSFT, etc.).
Format the response as a simple comma-separated list of tickers.

Data: {}
"""

ticker_response = Settings.llm.complete(ticker_prompt.format(str(result)))

# Convert the response into a list of tickers
tickers = [ticker.strip() for ticker in ticker_response.text.split(',') if ticker.strip()]

print("Extracted tickers:", tickers)

# Create a prompt to extract allocation values
allocation_prompt = """
Please analyze this data and extract the total value allocated to each asset or stock.
Present the information as a dictionary format with the asset name as the key and the allocated value as the number.
Only include entries that have clear monetary values allocated to them.

Data: {}
"""

allocation_response = Settings.llm.complete(allocation_prompt.format(str(result)))

print("\nAllocation Analysis:")
print(allocation_response.text)

# Convert the response into a dictionary (assuming the LLM returns a valid Python dictionary format)
try:
    # Using ast.literal_eval is safer than eval() for converting string to dict
    import ast
    allocations = ast.literal_eval(allocation_response.text)
    
    # Print formatted allocations
    print("\nFormatted Allocations:")
    for asset, value in allocations.items():
        print(f"{asset}: {value:,.2f} SEK")
        
except (ValueError, SyntaxError) as e:
    print("Could not parse allocations into dictionary format. Raw response shown above.")



Extracted tickers: ['Coinbase Global Ae', 'Valour Bitcoin (BTC) Zero SEK', 'Valour Ethereum (ETH) Zero SEK', 'Valour Solana (SOL) SEK']

Allocation Analysis:
{
    'Coinbase Global Ae': 1521507,
    'Swedbank Robur Technology A': 1168562,
    'Valour Bitcoin (BTC) Zero SEK': 2014676,
    'Valour Ethereum (ETH) Zero SEK': 1266073,
    'Valour Solana (SOL) SEK': 1466561
}

Formatted Allocations:
Coinbase Global Ae: 1,521,507.00 SEK
Swedbank Robur Technology A: 1,168,562.00 SEK
Valour Bitcoin (BTC) Zero SEK: 2,014,676.00 SEK
Valour Ethereum (ETH) Zero SEK: 1,266,073.00 SEK
Valour Solana (SOL) SEK: 1,466,561.00 SEK
